In [5]:
import torch
from torch.nn import functional as F
import numpy as np

## [FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/abs/2205.14135)

### Forward Pass

In [168]:
'''
    Q: (batch, seq_len, dim)
    K: (batch, seq_len, dim)
    V: (batch, seq_len, dim)
    tau: softmax scaling
    is_causal: whether to use causal mask
    dropout_p: dropout probability
    
    return: (batch, seq_len, dim)
'''

def flash_attention(Q, K, V, M = 100_000, tau = 1, dropout_p = 0, is_causal = False):
    b, N, d = Q.shape
    Bc = M // (4 * d)
    Br = min(Bc,d)
    O = torch.zeros_like(Q)
    l = torch.zeros(b, N)
    m = torch.ones(b, N) * -float('inf')
    Tr = N // Br + (1 if N % Br != 0 else 0)
    Tc = N // Bc + (1 if N % Bc != 0 else 0)

    for b_i in range(b):
        for j in range(Tc):
            N_c = Bc * j
            K_j = K[b_i,N_c:N_c + Bc]
            V_j = V[b_i,N_c:N_c + Bc]
            
            for i in range(Tr):
                N_r = Br * i
                Q_i = Q[b_i, N_r:N_r + Br]
                O_i = O[b_i, N_r:N_r + Br]
                l_i = l[b_i, N_r:N_r + Br]
                m_i = m[b_i, N_r:N_r + Br]

                S_ij = torch.matmul(Q_i, K_j.T) * tau
                
                if is_causal:
                    # Apply causal mask
                    r_max = N_c + Bc - N_r - 1
                    c_start = Bc - r_max
                    for r in range(min(r_max, S_ij.shape[0])):
                        S_ij[r, max(c_start + r, 0):] = torch.tensor(-float('inf'))

                m_ij = torch.max(S_ij, dim=1).values
                S_ij = S_ij - m_ij.unsqueeze(1)
                # Need to make sure that the exp is not nan, value of nan could be caused by applying causal mask
                S_ij[S_ij.isnan()] = 0
                P_ij = torch.exp(S_ij)
                # P_ij = torch.exp(torch.where(S_ij - m_ij.unsqueeze(1) != float('nan'), ))
                l_ij = torch.sum(P_ij, dim=1)                
                m_i_new = torch.max(m_i, m_ij)
                l_i_new = torch.exp(m_i - m_i_new) * l_i + torch.exp(m_ij - m_i_new) * l_ij
                if dropout_p > 0:
                    P_ij = F.dropout(P_ij, p = dropout_p, training = True)
                
                O_i = torch.diag(1 / l_i_new) @ (torch.diag(l_i * torch.exp(m_i - m_i_new)) @ O_i
                                                 + torch.diag(torch.exp(m_ij - m_i_new)) @ torch.matmul(P_ij, V_j))

                O[b_i, N_r:N_r + Br] = O_i
                l[b_i, N_r:N_r + Br] = l_i_new
                m[b_i, N_r:N_r + Br] = m_i_new
               
    return O

def attention(Q, K, V, tau = 1, dropout_p = 0, is_causal = False):
    S = (Q @ K.transpose(1,2)) * tau
    if is_causal:
        S = torch.where(torch.tril(torch.ones_like(S)) != 0, S, torch.tensor(-float('inf')))  # apply causal mask
    P = torch.softmax(S, dim=-1)
    if dropout_p > 0:
        P = F.dropout(P, p = dropout_p, training = True)

    O = P @ V

    return O

In [170]:
%%time


## Test

b, n, d = 16, 1024, 512
# b, n, d = 8, 128, 32

q=torch.randn(b, n, d)
k=torch.randn(b, n, d)
v=torch.randn(b, n, d)

o_a = flash_attention(q, k, v, tau=1/np.sqrt(d), is_causal = True)
o = attention(q, k, v, tau=1/np.sqrt(d), is_causal = True)

print(o_a[0][0][:30])
print(o[0][0][:30])
print(f'Total difference: {torch.sum(torch.abs(o_a - o))}')
print(o_a.allclose(o, atol=1e-4))

o_a = flash_attention(q, k, v, tau=1/np.sqrt(d), is_causal = True, dropout_p = 0.1)
print(o_a[0][0][:30])

tensor([ 1.4263, -0.8752,  1.2028,  0.3551,  0.4692, -0.6110, -0.0530,  0.4354,
        -0.3878, -1.3846,  0.2785,  0.4763, -0.0973,  0.2934,  0.0785, -1.0716,
        -0.0894, -0.1857, -0.7917,  0.3861,  0.2993,  1.3189,  1.6312, -0.7639,
        -0.4453,  0.5761,  0.0612,  1.8182,  0.4532, -0.5557])
tensor([ 1.4263, -0.8752,  1.2028,  0.3551,  0.4692, -0.6110, -0.0530,  0.4354,
        -0.3878, -1.3846,  0.2785,  0.4763, -0.0973,  0.2934,  0.0785, -1.0716,
        -0.0894, -0.1857, -0.7917,  0.3861,  0.2993,  1.3189,  1.6312, -0.7639,
        -0.4453,  0.5761,  0.0612,  1.8182,  0.4532, -0.5557])
Total difference: 0.3083489239215851
True
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])
CPU times: user 1min 3s, sys: 380 ms, total: 1min 3s
Wall time: 6.48 s


### Backward Pass

In [ ]:
def flash_attention_grad():
    pass

def attention_grad():
    pass